# Writing Continuous Applications with Structured Streaming Python APIs in Apache Spark

#### Tutorial for PyBay,  San Francisco

At first glance, building a distributed streaming engine might seem as simple as launching a set of servers and pushing data between them. Unfortunately, distributed stream processing runs into multiple complications that don’t affect simpler computations like batch jobs. Fortunately, PySpark 2.4 and Databricks makes this simple!

**Orignal Author**: Michael John

Modified and Ported to PySpark by Jules S. Damji for the Tutorial

In [2]:
1 + 1

Out[ 2 ]: 2

## Setup Data

In [4]:
%run "./setup/setup_data"

### PySpark Documentation: [Click here](https://spark.apache.org/docs/latest/api/python/index.html)
 * [DataFrame](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=dataframe#pyspark.sql.DataFrame)
 * [Spark SQL](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.types)
 * [Spark SQL Functions](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions)

# Processing Streaming Sensor Data

Structured Streaming is a powerful capability for building end-to-end continuous applications. At a high-level, it offers the following features:

1. __Output tables are always consistent__ with all the records in a prefix (partition) of the data, we will process and count in order.
1. __Fault tolerance__ is handled holistically by Structured Streaming, including in interactions with output sinks.
1. Ability to handle __late and out-of-order event-time data__. 

<img src="https://databricks.com/wp-content/uploads/2019/04/streaming_continuous-apps-1024x366.png" alt="" width="40%"/>

<sub>reference [Structured Streaming Blog](https://databricks.com/blog/2016/07/28/structured-streaming-in-apache-spark.html)</sub>

## 1-Sources

Consider the input data stream as the “Input Table”. Every data item that is arriving on the stream is like a new row being appended to the Input Table.

<img src="https://demo.cloud.databricks.com/files/mjohns/streaming/cloudtrail-unbounded-tables.png" alt="" width="40%"/>

## 2-Continuous Processing & Queries

The developer then defines a query on this source, or input table, _as if it were a static table_ to compute a final result table that will be written to an output sink. Spark automatically converts this batch-like query to a streaming execution plan. This is called incrementalization: Spark figures out what state needs to be maintained to update the result each time a record arrives. Finally, developers specify triggers to control when to update the results. Each time a trigger fires, Spark checks for new data (new row in the input table), and incrementally updates the result.

<img src="https://databricks.com/wp-content/uploads/2019/04/streaming_cloudtrail-unbounded-tables.png" alt="" width="40%"/>

## 3-Sinks

The last part of the model is output modes. Each time the result table is updated, the developer wants to write the changes to an external system, such as S3, HDFS, or a database. We usually want to write output incrementally. For this purpose, Structured Streaming provides three output modes:

* __Append__: Only the new rows appended to the result table since the last trigger will be written to the external storage. 
* __Complete__: The entire updated result table will be written to external storage, e.g. for aggregates.
* __Update__: Only the rows that were updated in the result table since the last trigger will be changed in the external storage. 

<img src="https://databricks.com/wp-content/uploads/2019/04/treaming_stream-example1-phone-updated.png" width="40%"/>

## A Continuous Application Example using PySpark Structured Streaming APIs

Setup some file paths our S3 bucket for output, checkpoint, and bad records

In [12]:
output_path = "/tmp/pydata/Streaming/continuous_streaming/out/iot-stream/"
checkpoint_path = "/tmp/pydata/Streaming/continuous_streaming/out/iot-stream-checkpoint"
#
#create checkpoint  path
#
dbutils.fs.rm(checkpoint_path,True) #overwrite checkpoint
dbutils.fs.mkdirs(checkpoint_path)
#
#
bad_records_path = "/tmp/pydata/Streaming/continuous_streaming/badRecordsPath/streaming-sensor/"
dbutils.fs.rm(bad_records_path, True) #empty dir
dbutils.fs.mkdirs(bad_records_path)

Out[ 4 ]: True

### What does the data from the sensors look like?

In [14]:
sensor_path = "/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/"
sensor_file_name= sensor_path + "streaming-sensor_file-1.json"
dbutils.fs.head(sensor_file_name, 233)

[Truncated to first 233 bytes]
 Out[ 5 ]: '{"timestamp":"2016-08-03 01:32:20.454","deviceId":44,"deviceType":"SensorTypeD","signalStrength":0.6627327869584749}\n{"timestamp":"2016-08-03 01:32:30.579","deviceId":63,"deviceType":"SensorTypeB","signalStrength":0.6419303214810813}'

### Define schemas for incoming stream and outgoing stream

Good best practice to define a schema and not have Spark infer it, for performance reasons. 
Without a schema, Spark will launch couple of jobs: one to read header, and another to read
a good chunck of the partition to validate the schema, ensuring it matches the data.

Now there are options that you can specifiy so that it fails fast, is tolerant, or subsitute missing
 values or mismatch dataytpes with NaN or null.

In [16]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

#original input schema
jsonSchema = (
  StructType()
  .add("timestamp", TimestampType()) #event time at the source
  .add("deviceId", LongType())
  .add("deviceType", StringType())
  .add("signalStrength", DoubleType())
)
# modified schema with added columns since we are 
# doing some ETL (transforming and adding extra columns)
# this transformed data will be stored into parquet files
# from which an SQL table can be created for consumption or
# report generation
parquetSchema = (
  StructType()
  .add("timestamp", TimestampType()) #event time at the source
  .add("deviceId", LongType())
  .add("deviceType", StringType())
  .add("signalStrength", DoubleType())
  .add("INPUT_FILE_NAME", StringType()) #file name from which this data item was read
  .add("PROCESSED_TIME", TimestampType())) #time at the executor while processing

### Read Stream from Object Store Source

In this case we're simulating a Kafka live stream by reading in a file at a time. But this could as well be Apache Kafka topics

__Notice: Intentionally slowing this down for tutorial.__

In [18]:
inputDF = ( spark 
          .readStream 
          .schema(jsonSchema) 
          .option("maxFilesPerTrigger", 1)  #slow it down for tutorial
          .option("badRecordsPath", bad_records_path) #any bad records will go here
          .json(sensor_path) #the source
          .withColumn("INPUT_FILE_NAME", input_file_name()) #maintain file path
          .withColumn("PROCESSED_TIME", current_timestamp()) #add a processing timestamp at the time of processing
          .withWatermark("PROCESSED_TIME", "1 minute") #optional: window for out of order data
         )

### Write Stream to Parquet File Sink

In [20]:
query = (inputDF
         .writeStream
         .format("parquet") #our sink to save it for posterity or batch queries if needed
         .option("path", output_path)
         .option("checkpointLocation", checkpoint_path) # add checkpointing for resiliency
         .outputMode("append")
         .queryName("devices") #optionally a query name over write to issue queries against
         .trigger(processingTime='5 seconds')
         .start() 
        )

In [21]:
%fs ls /tmp/pydata/Streaming/continuous_streaming/out/iot-stream/

path,name,size
dbfs:/tmp/pydata/Streaming/continuous_streaming/out/iot-stream/_spark_metadata/,_spark_metadata/,0
dbfs:/tmp/pydata/Streaming/continuous_streaming/out/iot-stream/part-00000-033d6ead-161f-4b57-a45e-bdfadcbe81dc-c000.snappy.parquet,part-00000-033d6ead-161f-4b57-a45e-bdfadcbe81dc-c000.snappy.parquet,4678
dbfs:/tmp/pydata/Streaming/continuous_streaming/out/iot-stream/part-00000-08935382-a904-4193-9ae8-755ab0b16c3b-c000.snappy.parquet,part-00000-08935382-a904-4193-9ae8-755ab0b16c3b-c000.snappy.parquet,4631
dbfs:/tmp/pydata/Streaming/continuous_streaming/out/iot-stream/part-00000-0a1b8a8d-4856-4b5c-8d91-274519bd886f-c000.snappy.parquet,part-00000-0a1b8a8d-4856-4b5c-8d91-274519bd886f-c000.snappy.parquet,4702
dbfs:/tmp/pydata/Streaming/continuous_streaming/out/iot-stream/part-00000-10ee49b8-914d-4a88-a0bc-92169a09a929-c000.snappy.parquet,part-00000-10ee49b8-914d-4a88-a0bc-92169a09a929-c000.snappy.parquet,4703
dbfs:/tmp/pydata/Streaming/continuous_streaming/out/iot-stream/part-00000-136039db-bf0b-4766-a22d-982cd98c48c9-c000.snappy.parquet,part-00000-136039db-bf0b-4766-a22d-982cd98c48c9-c000.snappy.parquet,4648
dbfs:/tmp/pydata/Streaming/continuous_streaming/out/iot-stream/part-00000-1f2ea9a9-1f83-4539-a847-aea2222f46e6-c000.snappy.parquet,part-00000-1f2ea9a9-1f83-4539-a847-aea2222f46e6-c000.snappy.parquet,4657
dbfs:/tmp/pydata/Streaming/continuous_streaming/out/iot-stream/part-00000-2b78b8c0-6154-438b-8624-7cc416e712c8-c000.snappy.parquet,part-00000-2b78b8c0-6154-438b-8624-7cc416e712c8-c000.snappy.parquet,4701
dbfs:/tmp/pydata/Streaming/continuous_streaming/out/iot-stream/part-00000-2c73afa3-5797-43f2-b8cc-b8e887652b54-c000.snappy.parquet,part-00000-2c73afa3-5797-43f2-b8cc-b8e887652b54-c000.snappy.parquet,4649
dbfs:/tmp/pydata/Streaming/continuous_streaming/out/iot-stream/part-00000-3cc302c3-bad1-4527-bb8d-9f323bf19f4b-c000.snappy.parquet,part-00000-3cc302c3-bad1-4527-bb8d-9f323bf19f4b-c000.snappy.parquet,4666


#### Create a temporary table from the input stream so that you can quickly issue SQL queries against it.

In [23]:
inputDF.createOrReplaceTempView("parquet_sensors")

### Run queries againt the temp table created off the input stream

In [25]:
%sql select * from parquet_sensors where deviceType = 'SensorTypeD' or deviceType = 'SensorTypeA'

timestamp,deviceId,deviceType,signalStrength,INPUT_FILE_NAME,PROCESSED_TIME
2016-08-03T01:32:20.454+0000,44,SensorTypeD,0.6627327869584749,dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-1.json,2019-08-15T18:02:56.201+0000
2016-08-03T01:32:11.230+0000,36,SensorTypeD,0.8869519587040092,dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-1.json,2019-08-15T18:02:56.201+0000
2016-08-03T01:32:29.490+0000,6,SensorTypeA,0.621775542316748,dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-1.json,2019-08-15T18:02:56.201+0000
2016-08-03T01:32:26.601+0000,31,SensorTypeA,0.7217142790760191,dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-1.json,2019-08-15T18:02:56.201+0000
2016-08-03T01:32:14.999+0000,2,SensorTypeD,0.9463019453554669,dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-1.json,2019-08-15T18:02:56.201+0000
2016-08-03T01:32:19.582+0000,31,SensorTypeA,0.4177194090879053,dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-1.json,2019-08-15T18:02:56.201+0000
2016-08-03T01:32:14.446+0000,61,SensorTypeA,0.8093833461353955,dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-1.json,2019-08-15T18:02:56.201+0000
2016-08-03T01:32:12.688+0000,93,SensorTypeA,0.6009808303441028,dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-1.json,2019-08-15T18:02:56.201+0000
2016-08-03T01:32:13.518+0000,45,SensorTypeD,0.705299532046275,dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-1.json,2019-08-15T18:02:56.201+0000
2016-08-03T01:32:15.719+0000,68,SensorTypeD,0.39902827917907346,dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-1.json,2019-08-15T18:02:56.201+0000


(Click Run All Above Here)

(Then Individually run below)

### Run queries and additional processing against Parquet file stored from the input stream

In [28]:
spark.conf.set("spark.sql.shuffle.partitions", "1") #keep the size of shuffles small for better query performance
devices = (spark.readStream
           .schema(parquetSchema)
           .format("parquet")
           .option("maxFilesPerTrigger", 1) #slow it down to demo
           .load(output_path)
           .withWatermark("PROCESSED_TIME", "1 minute") #window for out of order data
          )

 # generate temp table for more complex aggregation queries
devices.createOrReplaceTempView("sensors")

#### What files are being processed?

In [30]:
display(
  devices.
  select("INPUT_FILE_NAME", "PROCESSED_TIME")
  .groupBy("INPUT_FILE_NAME", "PROCESSED_TIME")
  .count()
  .orderBy("PROCESSED_TIME", ascending=False)
)

INPUT_FILE_NAME,PROCESSED_TIME,count
dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-99.json,2019-08-15T18:06:40.876+0000,100
dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-98.json,2019-08-15T18:06:35.541+0000,100
dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-97.json,2019-08-15T18:06:30.537+0000,100
dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-96.json,2019-08-15T18:06:25.468+0000,100
dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-95.json,2019-08-15T18:06:20.592+0000,100
dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-94.json,2019-08-15T18:06:15.635+0000,100
dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-93.json,2019-08-15T18:06:10.525+0000,100
dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-92.json,2019-08-15T18:06:05.589+0000,100
dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-91.json,2019-08-15T18:06:00.603+0000,100
dbfs:/mnt/jules-pydata/Streaming/continuous_streaming/streaming_sensor/streaming-sensor_file-90.json,2019-08-15T18:05:55.616+0000,100


#### How much data is coming through?

In [32]:
%sql select count(*) from sensors

count(1)
10000


#### What does the min, max, and average strength look like for each sensor type?

Use Spark SQL functions min(), max(), and avg()

Note: I can use SQL within my Python notebook via `%sql` magic command

In [34]:
%sql 

select count(*), deviceType, min(signalStrength), max(signalStrength), avg(signalStrength) 
  from sensors 
    group by deviceType 
    order by deviceType asc

count(1),deviceType,min(signalStrength),max(signalStrength),avg(signalStrength)
2505,SensorTypeA,3.5972269192152684E-4,0.9995903077803151,0.5039735263517676
2465,SensorTypeB,3.522297455231804E-4,0.9993696998766687,0.4977416083853139
2556,SensorTypeC,6.65677326306735E-4,0.999441839444791,0.5064546450953427
2474,SensorTypeD,2.412764460101302E-4,0.9997615682074666,0.5048238617609497


#### Let's create a stream to aggregate signal counts by device and timestamp over 5 second windows.

Note: this is tumbling window, not a sliding window; its size is 5 seconds

For example, discounting day, hours, and min, a tumbling window of size 5 looks as follows:

*[(00:00 - 00:05), (00:05: 00:10), (00:10: 00:15)]*

An event could fall into any of these tumbling windows.

In [36]:
(devices
 .groupBy(
   window("timestamp", "5 seconds"),
   "deviceId"
 )
 .count()
 .createOrReplaceTempView("sensor_counts")) #create a temporary view atop DataFrame

#### Which devices are experiencing signal loss over those 5 second windows?

In [38]:
%sql select * from sensor_counts where count < 5 order by window.start desc

window,deviceId,count
"List(2016-08-03T03:11:30.000+0000, 2016-08-03T03:11:35.000+0000)",23,1
"List(2016-08-03T03:11:30.000+0000, 2016-08-03T03:11:35.000+0000)",2,1
"List(2016-08-03T03:11:30.000+0000, 2016-08-03T03:11:35.000+0000)",82,1
"List(2016-08-03T03:11:30.000+0000, 2016-08-03T03:11:35.000+0000)",78,2
"List(2016-08-03T03:11:30.000+0000, 2016-08-03T03:11:35.000+0000)",98,1
"List(2016-08-03T03:11:25.000+0000, 2016-08-03T03:11:30.000+0000)",6,1
"List(2016-08-03T03:11:25.000+0000, 2016-08-03T03:11:30.000+0000)",86,2
"List(2016-08-03T03:11:25.000+0000, 2016-08-03T03:11:30.000+0000)",12,1
"List(2016-08-03T03:11:25.000+0000, 2016-08-03T03:11:30.000+0000)",89,1
"List(2016-08-03T03:11:25.000+0000, 2016-08-03T03:11:30.000+0000)",31,2


In [39]:
// --- SHOW DASHBOARD ---

File "<command-2704061155369639>" , line 1 
 (/, ---, SHOW, DASHBOARD, ---) 
 ^ 
 SyntaxError : invalid syntax

### Send Alerts for potentially down sensors that have not sent signals

Let's create a DataFrame from our temporary table `sensor_counts`

In [41]:
lost_sensor_signals = (spark.table("sensor_counts")
         .filter(col("count") < 5)
         .select("window.start", "window.end", "deviceId", "count")
         )

#display our DataFrame
display(lost_sensor_signals)

start,end,deviceId,count
2016-08-03T02:13:25.000+0000,2016-08-03T02:13:30.000+0000,81,1
2016-08-03T02:14:15.000+0000,2016-08-03T02:14:20.000+0000,59,1
2016-08-03T01:55:15.000+0000,2016-08-03T01:55:20.000+0000,92,2
2016-08-03T01:32:25.000+0000,2016-08-03T01:32:30.000+0000,52,1
2016-08-03T02:03:10.000+0000,2016-08-03T02:03:15.000+0000,42,1
2016-08-03T01:59:20.000+0000,2016-08-03T01:59:25.000+0000,70,1
2016-08-03T02:00:25.000+0000,2016-08-03T02:00:30.000+0000,16,1
2016-08-03T01:59:20.000+0000,2016-08-03T01:59:25.000+0000,79,1
2016-08-03T02:04:10.000+0000,2016-08-03T02:04:15.000+0000,2,1
2016-08-03T01:43:25.000+0000,2016-08-03T01:43:30.000+0000,91,1


##### Using `foreach` mechanism to write to workers' logs.

This could be used for monitoring purposes. Either another job scanning the logs for ALERTS
and publishing them onto a topic on Kafka or posting them on Ganglia. A good excerice to try
if you have a Kafka cluster aviable or Ganglia service available via a REST API.

In [43]:
def processRow(row):
  # for now write them to log files, but this logic can easily be extended to publishing alerts
  # to a topic on Kafka or monitoring/paging service such as Ganglia or PagerDuty.
  print("ALERT from Sensors: Between {} and {}, device {} reported only {} times".format(row.start, row.end, row.deviceId, row[3]))
  
(lost_sensor_signals
 .writeStream
 .outputMode("complete") #could be our Kafka topic "alerts" for monitoring
 .foreach(processRow)
 .start()
)

Out[ 17 ]: <pyspark.sql.streaming.StreamingQuery at 0x7fdbabf3c978>

_Here is an example of the results being written out._

<img src="https://databricks.com/wp-content/uploads/2018/12/Screen-Shot-2018-12-18-at-5.53.39-PM.png" alt="" width="50%"/>

##### CHALLENGE-1: 

Try using`foreachBatch` to write each micro-batch using standard DataFrame API.

__See [docs](https://docs.databricks.com/spark/latest/structured-streaming/foreach.html).__

##### CHALLENGE-2: 

Try adding adding additional monitoring conditions on devices. For example, add another streaming query that
filters monitoring conditions (signal strength < 0.2) from any of the above streams or tables: sensor_counts, devices, or inputStream, or parquet files.

__See [docs](https://docs.databricks.com/spark/latest/structured-streaming/foreach.html).__

## Cleanup Data

### Get all current running queries

In [49]:
sqm = spark.streams
[q.name for q in sqm.active]

### Stop them

In [51]:

[q.stop() for q in sqm.active]

In [52]:
%run "./setup/cleanup_data"